In [1]:
import ipynb.fs.full.get_news_headlines as gethl
import ipynb.fs.full.get_article_content as getart
import ipynb.fs.full.open_ai_models as openaimodel
import ipynb.fs.full.write_data as gswd
import ipynb.fs.full.read_data as gsrd
import urllib
import requests
import pandas
import json
import os
import sys
import pprint as pp
import dotenv as env
env.load_dotenv('../keys.env')

#<script async src="https://cse.google.com/cse.js?cx=314b1cf0785af4af3">
#</script>
#<div class="gcse-search"></div>

#API Doc here: https://developers.google.com/custom-search/v1/reference/rest/v1/cse/list

True

In [2]:
class google_Search:
    __SEARCH_API_KEY: str
    __cx_id: str
    
    def __init__(self):
        self.__SEARCH_API_KEY = os.environ.get('SEARCH_API_KEY')
        self.__cx_id = os.environ.get('SEARCH_CX_ID') 

    def __sanitize_query(self, query):
        query = query.strip()
        sanitized_query = urllib.parse.quote_plus(query)
        return sanitized_query
    
    def search(self, query, terms='None', num_pages=1, silent_mode=True):
        if(silent_mode):
            original_stdout = sys.stdout   # save the original stdout
            sys.stdout = open(os.devnull, 'w')
        headlines =[]
        links = []
        link_id = []
        max_page = 10   #max pages to which we can run custom search
        if(query!=''):
            query = self.__sanitize_query(query)
            cur_page = 0
            while cur_page < num_pages and cur_page < max_page :
                url = f'https://www.googleapis.com/customsearch/v1?key={self.__SEARCH_API_KEY}&cx={self.__cx_id}&q={query}&exact={terms}&num=10&start={cur_page*10+1}'
                print(f'Running search for {query} on page: {cur_page+1}')
                response = requests.get(url)
                if(response.status_code == 200):
                    data = response.json()
                    print(f'Found {len(data['items'])} on page {cur_page+1}')
                    for item in data['items']:
                        headlines.append(item['pagemap']['metatags'][0]['og:description']) #index called 'og:description  has a name of the link'
                        links.append(item['link'])
                        link_id.append(item['link'][-19:][:14])
                else:
                    print(f'Error: {response.status_code} at run page: {cur_page+1}')
                    break
                cur_page += 1 
            print('Combining all outcomes...')
            search_headlines = pandas.DataFrame({
                                    #'uuid': uuids, 
                                    'title': headlines,     #Don't change these column names, they are important for merging later 
                                    'link': links,
                                    'link_id': link_id
                                    })
            search_headlines.insert(0, 'site', 'Livemint')  #Because currently our search engine is set to search on LiveMint.
            
            print('Deduping links...')
            search_headlines.drop_duplicates(subset=['link_id'], keep='first', inplace=True, ignore_index=True)
            if(silent_mode):
                sys.stdout.close()
                sys.stdout = original_stdout    #restore the original stdout, other wise it will fail other prints.
            return search_headlines
        else:
            return None

In [4]:
#one line tester

#search_engine = google_Search()
#links = search_engine.search(query='Sumeet Bagadia', terms='Buy', num_pages=1, silent_mode=True)
#pp.pprint(links)

In [5]:
classified_hl = pandas.DataFrame(json.loads(gethl.classify_headlines(search_headlines['title'])))
classified_titles = search_headlines.join(classified_hl, lsuffix='_orig', rsuffix='_copy')   #make one df given with surety that matching is happening on indexes because Title matching used to give missed matches.. 
classified_titles = classified_titles[classified_titles.classification]
pp.pprint(classified_titles)

NameError: name 'search_headlines' is not defined

In [3]:
model = openaimodel.OpenAIModels()
response = model.classify_headlines(search_headlines['title'])
pandas.DataFrame(json.loads(response)['output'])

,title,classification,explanation
0,Sumeet Bagadia recommended buying these seven ...,True,Contains 'recommended buying' phrase.
1,Stocks to buy under ₹100: Motherson Sumi Wiri...,True,Contains 'suggested buys' phrase.
2,Breakout stocks to buy or sell: Sumeet Bagadia...,True,Contains 'recommends five shares to buy'.
3,Breakout stocks to buy or sell: Sumeet Bagadia...,True,Contains 'recommends five shares to buy'.
4,"Regarding stocks to buy under ₹100, Sumeet Ba...",True,Contains 'recommended buying' phrase.
5,Buy or sell: Sumeet Bagadia recommends three s...,True,Contains 'recommends three stocks to buy'.
6,Buy or sell: Sumeet Bagadia recommends these t...,True,Contains 'recommends these three stocks to buy'.
7,Buy or sell stocks: Sumeet Bagadia has recomme...,True,Contains 'recommended three stocks to buy'.
8,Breakout stocks to buy or sell: Sumeet Bagadia...,True,Contains 'recommends five shares to buy'.
